In [3]:
# IMPORT DES BIBLIOTHEQUES

import ta
import time
import math
import pprint
import asyncio
import datetime
import pandas as pd
import SecretConfig
import ccxt.pro as ccxt
from IPython.display import clear_output


In [4]:
# INITIALISATION DE L'EXCHANGE

exchange = ccxt.bitget({
    "apiKey":SecretConfig.APIKey,
    "secret":SecretConfig.SecretKey,
    'password': SecretConfig.Passphrase,
    "options": {'defaultType': 'swap', 'adjustForTimeDifference' : True},
    "timeout": 60000,
    "enableRateLimit": True,
})


In [5]:
# LIVE PRICE ET MARKETS

async def live_prices() :

    global last_20
    global tickers
    global swap_markets
  
    tickers = await exchange.fetch_tickers()
    timestamp = list(tickers.values())[0]['timestamp']
    index = [datetime.datetime.fromtimestamp(timestamp/1000).replace(microsecond=0)]

    data = {}
    for symbol, ticker in tickers.items():
        data[symbol +  " last"] = [ticker['last']]
        data[symbol +  " ask"] = [ticker['ask']]
        data[symbol +  " bid"] = [ticker['bid']]

    last_prices = pd.DataFrame(data, index=index)
    last_20 = pd.concat([last_20, last_prices])
    last_20 = last_20.tail(wind_EMA2 + wind_Var_EMA2 + wind_Div_EMA1_2 + wind_EMA2)

    markets = await exchange.load_markets()
    swap_markets = {symbol: market for symbol, market in markets.items() if market['swap']}

    # await asyncio.sleep(0.01)

    # clear_output(wait=True)
    # display(last_20)
    # return last_20


In [6]:
# CALCUL DES INDICATEURS TECHNIQUES

async def indicators_calculation() :
    
    global data1
    global data2
    global data3 
    global data4 
    global data5
    global data6
    global data7
    global data8

    global top_10_values

    global fulldata
       
    fulldata_row = pd.DataFrame()

    index = [last_20.index[-1]] 

    data_row_step1 = pd.DataFrame({symbol + suffix:
                                   [last_20[symbol + " last"].iloc[-1]] if suffix == " last" else 
                                   [last_20[symbol + " ask"].iloc[-1]] if suffix == " ask" else 
                                   [last_20[symbol + " bid"].iloc[-1]] if suffix == " bid" else 
                                   [((last_20[symbol + " last"].iloc[-1] - last_20[symbol + " last"].iloc[- wind_Var_Price]) /
                                     last_20[symbol + " last"].iloc[- wind_Var_Price]) * 100]  if suffix == " Var Last Price" and len(last_20) > wind_Var_Price else
                                     [ta.trend.ema_indicator(last_20[symbol + ' last'], window= wind_EMA1).iloc[-1]] if suffix == ' EMA_1' and len(last_20) > wind_EMA1 else
                                     [ta.trend.ema_indicator(last_20[symbol + ' last'], window= wind_EMA2).iloc[-1]] if suffix == ' EMA_2' and len(last_20) > wind_EMA2 else 
                                     [None] for symbol, ticker in tickers.items() for suffix in 
                                     [" last", " ask", " bid", ' Var Last Price', ' EMA_1', ' EMA_2']}, index=index)

                                
    data_row_step1 = data_row_step1.apply(pd.to_numeric)
    data1 = pd.concat([data1, data_row_step1])
    data1 = data1.tail(wind_Var_EMA1 + wind_Var_EMA2 + 1)
    # await asyncio.sleep(0.01)

   

    data_row_step2 = pd.DataFrame({symbol + suffix:
                                   (data1[symbol + " Var Last Price"].iloc[-1])**2 if suffix == ' Var Carre Last Price' and len(last_20) > wind_Var_Price else
                                   ((data1[symbol + ' EMA_1'].iloc[-1] - data1[symbol + ' EMA_1'].iloc[- wind_Var_EMA1]) /
                                    data1[symbol + ' EMA_1'].iloc[- wind_Var_EMA1]) * 100  if suffix == ' Var EMA_1' and len(last_20) > wind_EMA1 + wind_Var_EMA1  else
                                    ((data1[symbol + ' EMA_2'].iloc[-1] - data1[symbol + ' EMA_2'].iloc[- wind_Var_EMA2]) /
                                     data1[symbol + ' EMA_2'].iloc[- wind_Var_EMA2]) * 100  if suffix == ' Var EMA_2' and len(last_20) > wind_EMA2 + wind_Var_EMA2 else
                                     (data1[symbol + " last"].iloc[- 1] - data1[symbol + ' EMA_1'].iloc[-1]) ** 2 if suffix == ' Diff Carre Price_EMA_1' and len(last_20) > wind_EMA1 else
                                     (data1[symbol + ' EMA_1'].iloc[- 1] - data1[symbol + ' EMA_2'].iloc[-1]) ** 2 if suffix == ' Diff Carre EMA_1_2' and len(last_20) > wind_EMA2 else
                                     [None] for symbol, ticker in tickers.items() for suffix in 
                                     [' Var Carre Last Price', ' Var EMA_1', ' Var EMA_2', ' Diff Carre Price_EMA_1', ' Diff Carre EMA_1_2']}, index=index)
    
    data_row_step2 = data_row_step2.apply(pd.to_numeric)
    data2 = pd.concat([data2, data_row_step2])
    data2 = data2.tail(wind_Div_Price_EMA1 + wind_Div_EMA1_2 + 1)
    # await asyncio.sleep(0.01)
    
    data_row_step3 = pd.DataFrame({symbol + suffix:
                                   ((data2[symbol + ' Diff Carre Price_EMA_1'].iloc[-1] - data2[symbol + ' Diff Carre Price_EMA_1'].iloc[- wind_Div_Price_EMA1]) /
                                    data2[symbol + ' Diff Carre Price_EMA_1'].iloc[- wind_Div_Price_EMA1]) * 100  if suffix == ' Divergence Price_EMA_1' and len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 and data2[symbol + ' Diff Carre Price_EMA_1'].iloc[- wind_Div_Price_EMA1] != 0 else
                                    ((data2[symbol + ' Diff Carre EMA_1_2'].iloc[-1] - data2[symbol + ' Diff Carre EMA_1_2'].iloc[- wind_Div_EMA1_2]) /
                                    data2[symbol + ' Diff Carre EMA_1_2'].iloc[- wind_Div_EMA1_2]) * 100  if suffix == ' Divergence EMA_1_2' and len(last_20) > wind_EMA2 + wind_Div_EMA1_2 and data2[symbol + ' Diff Carre EMA_1_2'].iloc[- wind_Div_EMA1_2] != 0 else
                                    [None] for symbol, ticker in tickers.items() for suffix in 
                                    [' Divergence Price_EMA_1', ' Divergence EMA_1_2']}, index=index)
    
    data_row_step3 = data_row_step3.apply(pd.to_numeric)
    data3 = pd.concat([data3, data_row_step3])
    data3 = data3.tail(wind_Div_Moy_Price_EMA1 + wind_Div_Moy_EMA1_2 + 1)
    # await asyncio.sleep(0.01)

    data_row_step4 = pd.DataFrame({symbol + suffix:
                                   ta.trend.ema_indicator(data3[symbol + ' Divergence Price_EMA_1'], window= wind_Div_Moy_Price_EMA1).iloc[-1] if suffix == ' Divergence Moyen Price_EMA_1' and len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 else 
                                   ta.trend.ema_indicator(data3[symbol + ' Divergence EMA_1_2'], window= wind_Div_Moy_EMA1_2).iloc[-1] if suffix == ' Divergence Moyen EMA_1_2' and len(last_20) > wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2  else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Divergence Moyen Price_EMA_1', ' Divergence Moyen EMA_1_2']}, index=index)
    
    data_row_step4 = data_row_step4.apply(pd.to_numeric)
    data4 = pd.concat([data4, data_row_step4])
    data4 = data4.tail(wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2)
    # await asyncio.sleep(0.01)

    data_row_step5 = pd.DataFrame({symbol + suffix:
                                   data4[symbol + ' Divergence Moyen Price_EMA_1'].iloc[-1] > 0 and data4[symbol + ' Divergence Moyen EMA_1_2'].iloc[-1]  > 0 if suffix == ' Condition'
                                   and len(last_20) > wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 else
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Condition']}, index=index)
    
    # data_row_step5 = data_row_step5.apply(pd.to_numeric)
    data5 = pd.concat([data5, data_row_step5])
    # data5 = data5.apply(pd.to_numeric)
    data5 = data5.tail(5)
    # clear_output(wait=True)
    # display(data5.tail(2))
    # await asyncio.sleep(0.01)


    top_div = {}
    top_10_values = {}

    for symbol in tickers:
        
        if len(last_20) > wind_EMA1 + wind_Div_Price_EMA1:
            
            if data3[symbol + ' Divergence Price_EMA_1'].iloc[-1] > div_min_top:
                
                top_div[symbol] = data3[symbol + ' Divergence Price_EMA_1'].iloc[-1]

    if top_div :
        
        top_10_values = dict(sorted(top_div.items(), key=lambda item: item[1], reverse=True)[:10])

    # display(f'top is : {top_10_values}')
  
    data_row_step6 = pd.DataFrame({symbol + suffix:
                                   1 if suffix == ' Top_10' and top_10_values is not None and symbol in top_10_values.keys() and len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Top_10']}, index=index)
    
    data_row_step6 = data_row_step6.apply(pd.to_numeric)
    data6 = pd.concat([data6, data_row_step6])
    data6 = data6.tail(occur_wind)
    # clear_output(wait=True)
    '''if len(last_20) > wind_EMA1 + wind_Div_Price_EMA1 :
        display(data6.tail(2))'''
    # await asyncio.sleep(0.01)

    data_row_step7 = pd.DataFrame({symbol + suffix: 
                                   data6[symbol + ' Top_10'].sum() if suffix == ' Occur' and len(fulldata) > wind_EMA1 + wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 + occur_wind + 1 else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Occur']}, index=index)
    data_row_step7 = data_row_step7.apply(pd.to_numeric)
    data7 = pd.concat([data7, data_row_step7])
    data7 = data7.tail(2)
    '''if len(fulldata) > wind_EMA1 + wind_Div_Price_EMA1 + wind_Div_Moy_Price_EMA1 + occur_wind :
        display(data7)'''
    # await asyncio.sleep(0.01)

    data_row_step8 = pd.DataFrame({symbol + suffix:
                                   "Long" if suffix == ' Tradable' and data7[symbol +' Occur'].iloc[-1] is not None and data7[symbol +' Occur'].iloc[-1] >
                                     min_occur and data5[symbol + ' Condition'].iloc[-1] and
                                   data1[symbol + ' Var Last Price'].iloc[-1] > 0 and data1[symbol + " last"].iloc[-1] >
                                     data1[symbol + ' EMA_1'].iloc[-1] and len(fulldata) >
                                       wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind  else
                                   "Short" if suffix == ' Tradable' and data7[symbol +' Occur'].iloc[-1] is not None and data7[symbol +' Occur'].iloc[-1] >
                                     min_occur and data5[symbol + ' Condition'].iloc[-1] and
                                   data1[symbol + ' Var Last Price'].iloc[-1] < 0 and data1[symbol + " last"].iloc[-1] <
                                     data1[symbol + ' EMA_1'].iloc[-1] and len(fulldata) > wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind else 
                                   [None] for symbol, ticker in tickers.items() for suffix in 
                                   [' Tradable']}, index=index) 
    data8 = pd.concat([data8, data_row_step8])
    data8 = data8.tail(2)
    '''if len(fulldata) > wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind :
        display(data8)'''
    # await asyncio.sleep(0.01)


    fulldata_row = pd.concat([data_row_step1, data_row_step2, data_row_step3, data_row_step4, data_row_step5, data_row_step6, data_row_step7, data_row_step8], axis=1)
    fulldata_row = fulldata_row.apply(pd.to_numeric, errors='ignore')
    fulldata_row =fulldata_row.sort_index(axis=1)
    fulldata = pd.concat([fulldata, fulldata_row])
    fulldata = fulldata.tail(wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind + wind_EMA2)
    # clear_output(wait=True)
    # display(data_row_step8)
    # await asyncio.sleep(0.01)
    

In [7]:
# MISE A JOUR DE LA BALANCE

async def refresh_balance() :

    global balance
    global free_balance

    balance_data = await exchange.fetch_balance()
    balance = float(balance_data['info'][0]['available'])
    free_balance = float(balance_data['free']['USDT'])
    # display(f'Balance is {balance}')
    # await asyncio.sleep(0.01)

In [19]:
# PASSAGE DES ORDRES

async def trading() :

    global df_open_trade_info
    global df_closed_trade_info
    global nbr_open_trade
    global trade_nb
    global nbr_max_trade
    global trail_stop


    # balance = await exchange.fetch_balance()

    # display('trading init')

    # pprint.pprint('df open trade not empty')
    
    list_change_sl = []

    
    if not df_open_trade_info.empty :

        response = []

        for symbol in df_open_trade_info.index:
            
            response = await exchange.fetch_positions([symbol])
            await asyncio.sleep(0.005)

            if not response :

                if df_open_trade_info.loc[symbol, 'id'] == None and df_open_trade_info.loc[symbol,'sl_id'] != None :
                    
                    params = {"stop" : True}
                    response = await exchange.fetch_open_orders(symbol, params= params)
                    await asyncio.sleep(0.005)
                
                    if response :
                    
                        for resp in response :

                            if resp["id"] :

                                try : 
                                    
                                    params = {'stop' : True, "planType" : "normal_plan"}
                                    await exchange.cancel_order(resp["id"], symbol, params= params)

                                    df_open_trade_info.loc[symbol, 'status'] == "bad-config"

                                except Exception :
                                    
                                    df_open_trade_info.loc[symbol, 'status'] == "bad-config"

                elif df_open_trade_info.loc[symbol,'sl_id'] == None and df_open_trade_info.loc[symbol,'id'] != None :

                    params = {"stop" : True}
                    response = await exchange.fetch_open_orders(symbol, params= params)
                    await asyncio.sleep(0.005)
                
                    if response :
                    
                        for resp in response :

                            if resp["id"] :
                                
                                try : 
                                    
                                    params = {'stop' : True, "planType" : "normal_plan"}
                                    await exchange.cancel_order(resp["id"], symbol, params= params)

                                    df_open_trade_info.loc[symbol, 'status'] == "bad-config"

                                except Exception :
                                    
                                    df_open_trade_info.loc[symbol, 'status'] == "bad-config"
  
                else : 
                    
                    params = {"stop" : True}
                    response = await exchange.fetch_open_orders(symbol, params= params)
                    await asyncio.sleep(0.005)
                
                    if len(response) != 2 :
                    
                        for resp in response :

                            if resp["id"] :
                                
                                try : 
                                    
                                    params = {'stop' : True, "planType" : "normal_plan"}
                                    await exchange.cancel_order(resp["id"], symbol, params= params)

                                    df_open_trade_info.loc[symbol, 'status'] == "bad-config"

                                except Exception :
                                    
                                    df_open_trade_info.loc[symbol, 'status'] == "bad-config"

        list_change_sl = [
            {
                'symbol': symbol,
                "type": 'limit',
                'side': df_open_trade_info['sl_side'].loc[symbol] ,
                'amount': df_open_trade_info['amount'].loc[symbol],
                "price": exchange.price_to_precision(symbol, fulldata[symbol + ' bid'].iloc[-1] * (1 - trail_stop) - (swap_markets[symbol]['precision']['price'] * 2)) if
                df_open_trade_info['sl_side'].loc[symbol] == "close_long" else 
                exchange.price_to_precision(symbol, fulldata[symbol + ' ask'].iloc[-1] * (1 + trail_stop) + (swap_markets[symbol]['precision']['price'] * 2)),
                'params' :{
                    "marginCoin": "USDT",
                    'timeInForceValue': 'post_only',
                    "triggerPrice": exchange.price_to_precision(symbol, fulldata[symbol + ' bid'].iloc[-1] * (1 - trail_stop) - (swap_markets[symbol]['precision']['price'])) if
                    df_open_trade_info['sl_side'].loc[symbol] == "close_long" else
                    exchange.price_to_precision(symbol, fulldata[symbol + ' ask'].iloc[-1] * (1 + trail_stop) + (swap_markets[symbol]['precision']['price'])),
                    "triggerType":"market_price",
                    "reduceOnly" : True,
                    "leverage": "1",
                    "planType" : "normal_plan",
                    'clientOid': f'{df_open_trade_info['clientOid'].loc[symbol]}_slV_{df_open_trade_info["sl_version"].loc[symbol] + 1}'
                }
            } for symbol in df_open_trade_info.index if df_open_trade_info.loc[symbol, 'sl_side'] == "close_long" and float(
                df_open_trade_info.loc[symbol, 'sl_triggerPrice']) < float(
                    exchange.price_to_precision(symbol, fulldata[symbol + ' bid'].iloc[-1] * (1 - trail_stop) - (
                        swap_markets[symbol]['precision']['price']))) or df_open_trade_info.loc[symbol, 'sl_side'] == "close_short" and float(
                            df_open_trade_info.loc[symbol, 'sl_triggerPrice']) > float(exchange.price_to_precision(
                                symbol, fulldata[symbol + ' ask'].iloc[-1] * (1 + trail_stop) + (swap_markets[symbol]['precision']['price'])))] or df_open_trade_info.loc[symbol, 'status'] == "set_new_sl"
        
        await asyncio.sleep(0.005)
        
        # pprint.pprint(list_change_sl)
        # pprint.pprint(f'list_change_sl is ok')

        for symbol in df_open_trade_info.index:
            
            # pprint.pprint(f'last step for {symbol}')

            if list_change_sl and symbol in [item['symbol'] for item in list_change_sl] :

                for item in list_change_sl :

                    if item['symbol'] == symbol :

                        # pprint.pprint(f'Trying bloc sl')

                        if df_open_trade_info.loc[symbol, 'status'] != "set_new_sl" :

                            params = {"stop" : True}
                            response = await exchange.fetch_open_orders(symbol, params= params)
                            await asyncio.sleep(0.005)
                
                            if response :
                    
                                for resp in response :

                                    if resp["id"] and resp["info"]["side"] == "close_long" or resp["id"] and resp["info"]["side"] == "close_short" :

                                        params = {'stop' : True, "planType" : "normal_plan"}

                                        try : 

                                            params = {'stop' : True, "planType" : "normal_plan"}
                                            await exchange.cancel_order(resp["id"], symbol, params= params)

                                        except Exception :
                                            
                                            df_open_trade_info.loc[symbol, 'status'] == "bad-config"

                        if df_open_trade_info.loc[symbol, 'status'] != "bad-config" :
                            
                            new_sl = await exchange.create_order(symbol, item["type"], item['side'], item['amount'], item['price'], item['params'])
                        
                            await asyncio.sleep(0.01)

                            # pprint.pprint(f'last step condition1 for {symbol}')

                            df_open_trade_info.loc[symbol, 'sl_triggerPrice'] = float(item['params']['triggerPrice'])

                            df_open_trade_info.loc[symbol, 'sl_Execut_Price'] = float(item["price"])

                            df_open_trade_info.loc[symbol, "sl_id"] = new_sl["id"]

                            df_open_trade_info.loc[symbol, 'sl_clientOid'] = item['params']['clientOid']
                
                            df_open_trade_info.loc[symbol, 'sl_version'] += 1
            
            elif df_open_trade_info.loc[symbol, 'sl_side'] == "close_long" and fulldata[symbol + ' last'].iloc[-1] <= float(df_open_trade_info.loc[symbol, 'sl_triggerPrice']) :

                # pprint.pprint(f'last step condition2 for {symbol}')

                df_open_trade_info.loc[symbol, 'status'] = "close"

            elif df_open_trade_info.loc[symbol, 'sl_side'] == "close_short" and fulldata[symbol + ' last'].iloc[-1] >= float(df_open_trade_info.loc[symbol, 'sl_triggerPrice']) :

                # pprint.pprint(f'last step condition2 for {symbol}')

                df_open_trade_info.loc[symbol, 'status'] = "close"

        
        await asyncio.sleep(0.005)
        
        # pprint.pprint(df_open_trade_info)
        # pprint.pprint(f'Trying bloc time')

        for symbol in df_open_trade_info.index :

            response = []

            params = {'stop' : True}

            response = await exchange.fetch_open_orders(symbol, params= params)

            await asyncio.sleep(0.005)

            if response :

                # pprint.pprint(f'Trying Resp bloc time')

                for resp in response :

                    if resp['id'] and resp['info']['side'] == 'open_long' and  (
                        datetime.timedelta(seconds=round((fulldata.index[-1] - datetime.datetime.fromtimestamp(
                            resp['timestamp']/1000)).total_seconds()))).total_seconds() > time_to_fill or resp['id'] and resp['info']['side'] == 'open_short' and (
                                datetime.timedelta(seconds=round((fulldata.index[-1] - datetime.datetime.fromtimestamp(
                                    resp['timestamp']/1000)).total_seconds()))).total_seconds() > time_to_fill :
                        
                        try : 
                                    
                            params = {'stop' : True, "planType" : "normal_plan"}

                            await exchange.cancel_order(df_open_trade_info["sl_id"].loc[symbol], symbol, params= params)

                            await exchange.cancel_order(df_open_trade_info["id"].loc[symbol], symbol, params= params)

                            df_open_trade_info.loc[symbol, 'status'] == "cancel"

                            display(f'trade {symbol} close for time')

                        except Exception :

                            display(f'trade {symbol} close for time cause Exception')

                            df_open_trade_info.loc[symbol, 'status'] == "cancel"
                        
            await asyncio.sleep(0.005)

        for symbol in df_open_trade_info.index :

            # pprint.pprint(f'Trying drop symbol close or cancel')
            
            if df_open_trade_info.loc[symbol, 'status'] == "close" or df_open_trade_info.loc[symbol, 'status'] == "cancel" or df_open_trade_info.loc[symbol, 'status'] == "bad-config" :
                
                df_closed_trade_info = pd.concat([df_closed_trade_info, df_open_trade_info.loc[[symbol]]])
                df_open_trade_info = df_open_trade_info.drop(symbol)

                nbr_open_trade -= 1
        
        await refresh_balance()
        await asyncio.sleep(0.005)
        
        # list_change_sl = []
        
        # pprint.pprint(f'df_closed_trade_info is : !!!!!!!')
        # pprint.pprint(df_closed_trade_info)
        # pprint.pprint(f'df_open_trade_info is : !!!!!!!')
        # display(df_open_trade_info)


    if top_10_values is not None and len(fulldata) > wind_EMA1 + wind_EMA2 + wind_Div_EMA1_2 + wind_Div_Moy_EMA1_2 + occur_wind and not data8.empty and data8.iloc[-1].notnull().any() and free_balance > 50 and nbr_open_trade <= nbr_max_trade :

        # pprint.pprint('tentative top 10 pas vide')

        # for symbol, ticker in tickers.items() :
        for symbol in top_10_values.keys():
            
            trade = []
            response_create = []
            # pprint.pprint(f'tentative pour {symbol}')
            
            if symbol not in df_open_trade_info.index and data8[symbol + ' Tradable'].iloc[-1] == 'Long' :

                # pprint.pprint(f'tentative data8 pour {symbol}')

                trade = [
                    {
                        'symbol': symbol,
                        "type": 'limit',
                        'side': 'open_long',
                        'amount': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                        "price": exchange.price_to_precision(symbol, fulldata[symbol + ' ask'].iloc[-1] + (swap_markets[symbol]['precision']['price'] * 2)),
                        'params' : {
                            "marginCoin": "USDT",
                            'timeInForceValue': "post_only",
                            "triggerPrice": exchange.price_to_precision(symbol, fulldata[symbol + ' last'].iloc[-1] + (swap_markets[symbol]['precision']['price'])),
                            'triggerType': 'market_price',
                            "leverage": "1",
                            "planType" : "normal_plan",
                            'clientOid': f"Trade_nb_{trade_nb}"
                        }
                    },
                    {
                        'symbol': symbol,
                        "type": 'limit',
                        'side': 'close_long',
                        'amount': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                        "price": exchange.price_to_precision(symbol, fulldata[symbol + ' bid'].iloc[-1] * (1 - trail_stop) - (swap_markets[symbol]['precision']['price'] * 2)),
                        'params' :{
                            "marginCoin": "USDT",
                            'timeInForceValue': 'post_only',
                            "triggerPrice": exchange.price_to_precision(symbol, fulldata[symbol + ' last'].iloc[-1] * (1 - trail_stop) - (swap_markets[symbol]['precision']['price'])),
                            "triggerType":"market_price",
                            "reduceOnly" : True,
                            "leverage": "1",
                            "planType" : "normal_plan",
                            'clientOid': f"Trade_nb_{trade_nb}_slV_1"
                        }
                    }]

                id = None
                sl_id = None

                for signal in trade :

                    response_create = []
                    
                    await exchange.create_order(signal['symbol'], signal["type"], signal['side'], signal['amount'], signal['price'], signal['params'])
                
                    await asyncio.sleep(0.01)

                    params = {"stop" : True}
                    response_create = await exchange.fetch_open_orders(symbol, params= params)

                    await asyncio.sleep(0.005)

                    if response_create :
                        
                        for resp in response_create :
                            
                            if resp['info']['side'] == 'open_long' or resp['info']['side'] == 'open_short' :
                                
                                id = resp["id"]
                                
                            if resp['info']['side'] == 'close_long' or resp['info']['side'] == 'close_short' :
                                
                                sl_id = resp["id"]
                
                df_trade_row = pd.DataFrame({
                    'symbol': [trade[0]['symbol']],
                    'amount' : [float(trade[0]['amount'])],
                    'side': [trade[0]['side']],
                    'sl_side': [trade[1]['side']],
                    'opening_price': [float(trade[0]['price'])],
                    'sl_triggerPrice': [float(trade[1]['params']['triggerPrice'])],
                    'sl_Execut_Price': [float(trade[1]['price'])],
                    'clientOid' : [trade[0]['params']['clientOid']],
                    'sl_clientOid' : [trade[1]['params']['clientOid']],
                    'id': [id],
                    "sl_id" : [sl_id],
                    'status' : ["Open"],
                    'sl_version' : [1]
                    })
                    
                # pprint.pprint(df_trade_row)
                df_trade_row.set_index('symbol', inplace=True)
                # pprint.pprint(f'index is ok')
                # df_trade_row['sl_triggerPrice'] = df_trade_row['sl_triggerPrice'].apply(pd.to_numeric)
                df_open_trade_info = pd.concat([df_open_trade_info, df_trade_row])
                # pprint.pprint(df_open_trade_info)
                nbr_open_trade += 1
                trade_nb += 1 

                await refresh_balance()

            elif symbol not in df_open_trade_info.index and data8[symbol + ' Tradable'].iloc[-1] == 'Short':

                # pprint.pprint(f'tentative data8 pour {symbol}')
            
                trade = [
                    {
                        'symbol': symbol,
                        "type": 'limit',
                        'side': 'open_short',
                        'amount': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                        "price": exchange.price_to_precision(symbol, fulldata[symbol + ' bid'].iloc[-1] - (swap_markets[symbol]['precision']['price'] * 2)),
                        'params' : {
                            "marginCoin": "USDT",
                            'timeInForceValue': "post_only",
                            "triggerPrice": exchange.price_to_precision(symbol, fulldata[symbol + ' last'].iloc[-1] - (swap_markets[symbol]['precision']['price'])),
                            "triggerType":"market_price",
                            "leverage": "1",
                            "planType" : "normal_plan",
                            'clientOid': f"Trade_nb_{trade_nb}"
                        }
                    },
                    {
                        'symbol': symbol,
                        "type": 'limit',
                        'side': 'close_short',
                        'amount': exchange.amount_to_precision(symbol, round(balance * (0.9/nbr_max_trade), 2) / fulldata[symbol + ' last'].iloc[-1]),
                        "price": exchange.price_to_precision(symbol, fulldata[symbol + ' ask'].iloc[-1] * (1 + trail_stop) + (swap_markets[symbol]['precision']['price'] * 2)),
                        'params' : {
                            "marginCoin": "USDT",
                            "timeInForceValue": "post_only",
                            "triggerPrice": exchange.price_to_precision(symbol, fulldata[symbol + ' last'].iloc[-1] * (1 + trail_stop) + (swap_markets[symbol]['precision']['price'])),
                            "triggerType":"market_price",
                            "reduceOnly" : True,
                            "leverage": "1",
                            "planType" : "normal_plan",
                            'clientOid': f"Trade_nb_{trade_nb}_slV_1"
                        }
                    }]

                id = None
                sl_id = None

                for signal in trade :

                    response_create = []
                    
                    await exchange.create_order(signal['symbol'], signal["type"], signal['side'], signal['amount'], signal['price'], signal['params'])
                
                    await asyncio.sleep(0.005)

                    params = {"stop" : True}
                    response_create = await exchange.fetch_open_orders(symbol, params= params)

                    await asyncio.sleep(0.005)

                    if response_create :
                        
                        for resp in response_create :
                            
                            if resp['info']['side'] == 'open_long' or resp['info']['side'] == 'open_short' :
                                
                                id = resp["id"]
                                
                            if resp['info']['side'] == 'close_long' or resp['info']['side'] == 'close_short' :
                                
                                sl_id = resp["id"]
                
                df_trade_row = pd.DataFrame({
                    'symbol': [trade[0]['symbol']],
                    'amount' : [float(trade[0]['amount'])],
                    'side': [trade[0]['side']],
                    'sl_side': [trade[1]['side']],
                    'opening_price': [float(trade[0]['price'])],
                    'sl_triggerPrice': [float(trade[1]['params']['triggerPrice'])],
                    'sl_Execut_Price': [float(trade[1]['price'])],
                    'clientOid' : [trade[0]['params']['clientOid']],
                    'sl_clientOid' : [trade[1]['params']['clientOid']],
                    'id': [id],
                    "sl_id" : [sl_id],
                    'status' : ["Open"],
                    'sl_version' : [1]
                    })
                    
                # pprint.pprint(df_trade_row)
                df_trade_row.set_index('symbol', inplace=True)
                # pprint.pprint(f'index is ok')
                # df_trade_row['sl_triggerPrice'] = df_trade_row['sl_triggerPrice'].apply(pd.to_numeric)
                df_open_trade_info = pd.concat([df_open_trade_info, df_trade_row])
                # pprint.pprint(df_open_trade_info)
                nbr_open_trade += 1
                trade_nb += 1 

                await refresh_balance()

    # await asyncio.sleep(0.01)
    
    # if len(df_open_trade_info) > 0 :
      #  display(df_open_trade_info)
    # if len(df_closed_trade_info) > 0 :
      #  display(df_closed_trade_info)


    if balance <= 190 :
        # pprint.pprint(df_closed_trade_info)
        # pprint.pprint(df_open_trade_info)
        raise Exception            


In [21]:

nbr_max_trade = 15
time_to_fill = 45
min_occur = 3
div_min_top = 100000
trail_stop = 0.001

wind_Var_Price = 3
wind_EMA1 = 4
wind_Var_EMA1 = 3
wind_EMA2 = 9
wind_Var_EMA2 = 4
wind_Div_Price_EMA1 = 4
wind_Div_EMA1_2 = 6
wind_Div_Moy_Price_EMA1 = 2
wind_Div_Moy_EMA1_2 = 2

occur_wind = 20

trade_nb = 0
nbr_open_trade = 0
alertnbr = 1

data1 = pd.DataFrame()
data2 = pd.DataFrame()
data3 = pd.DataFrame()
data4 = pd.DataFrame()
data5 = pd.DataFrame()
data6 = pd.DataFrame()
data7 = pd.DataFrame()
data8 = pd.DataFrame()
last_20 = pd.DataFrame()
alert_df = pd.DataFrame()
fulldata = pd.DataFrame()
df_open_trade_info = pd.DataFrame()
df_closed_trade_info = pd.DataFrame()



async def main():

    global wind_Var_Price 
    global wind_EMA1 
    global wind_Var_EMA1
    global wind_EMA2
    global wind_Var_EMA2
    global wind_Div_Price_EMA1
    global wind_Div_EMA1_2
    global wind_Div_Moy_Price_EMA1
    global wind_Div_Moy_EMA1_2

    global occur_wind
    global min_occur
    global div_min_top
    global trail_stop
    global nbr_max_trade
    global nbr_open_trade
    global trade_nb
    global alert_df
    global alertnbr
    global time_to_fill

    global last_20

    await refresh_balance()

    while True:
        try:

            display(alert_df)
            display(f'Balance is {balance}')
            display(last_20.tail(3))
            display(df_closed_trade_info)
            display(df_open_trade_info)

            await live_prices()
            # await asyncio.sleep(0.01)
            await indicators_calculation()
            # await asyncio.sleep(0.01)

        except Exception as e:
            print('Error:', str(e))
            await exchange.close()
            break
        await asyncio.sleep(0.001)  # respect the rateLimit

        try:

            await trading()
            await asyncio.sleep(0.005)

        except Exception as e:

            alert = {f'alert_nbr {alertnbr}': str(e)}
            alert_data = pd.DataFrame(alert, index=[0]) 
            alert_df = pd.concat([alert_df, alert_data])
            alertnbr += 1

        clear_output(wait=True)

        await asyncio.sleep(0.001)  # respect the rateLimit

await main()

""


'Balance is 195.87223852'

,BTC/USDT:USDT last,BTC/USDT:USDT ask,BTC/USDT:USDT bid,ETH/USDT:USDT last,ETH/USDT:USDT ask,ETH/USDT:USDT bid,XRP/USDT:USDT last,XRP/USDT:USDT ask,XRP/USDT:USDT bid,EOS/USDT:USDT last,...,TIA/USDT:USDT bid,TWT/USDT:USDT last,TWT/USDT:USDT ask,TWT/USDT:USDT bid,MEME/USDT:USDT last,MEME/USDT:USDT ask,MEME/USDT:USDT bid,PYTH/USDT:USDT last,PYTH/USDT:USDT ask,PYTH/USDT:USDT bid
2023-12-10 06:11:49,43928.5,43928.9,43928.8,2353.67,2354.29,2354.27,0.66731,0.66740,0.66731,0.8532,...,9.5044,1.2587,1.2593,1.2584,0.039216,0.039245,0.039224,0.4618,0.4620,0.4616
2023-12-10 06:12:05,43928.9,43928.8,43926.5,2354.27,2354.28,2354.27,0.66709,0.66709,0.66698,0.8528,...,9.5048,1.2587,1.2592,1.2583,0.039226,0.039237,0.039214,0.4618,0.4621,0.4617
2023-12-10 06:12:20,43926.5,43926.6,43926.5,2354.52,2354.51,2354.50,0.66720,0.66720,0.66701,0.8528,...,9.5071,1.2589,1.2590,1.2583,0.039201,0.039245,0.039244,0.4619,0.4622,0.4618


,amount,side,sl_side,opening_price,sl_triggerPrice,sl_Execut_Price,clientOid,sl_clientOid,id,sl_id,status,sl_version
symbol,,,,,,,,,,,,
GMT/USDT:USDT,38.0,open_short,close_short,0.306600,0.30710,0.307500,Trade_nb_2,Trade_nb_2_slV_1,None,1117610081481465858,close,1
MANA/USDT:USDT,21.0,open_short,close_short,0.540800,0.54090,0.541000,Trade_nb_0,Trade_nb_0_slV_4,1117609901831032832,1117610106299162624,close,4
RSR/USDT:USDT,3835.0,open_short,close_short,0.003063,0.00307,0.003073,Trade_nb_3,Trade_nb_3_slV_1,1117610208430465024,1117610211739766786,close,1
PYTH/USDT:USDT,25.0,open_long,close_long,0.462200,0.46150,0.460900,Trade_nb_5,Trade_nb_5_slV_1,1117610314995142656,1117610318170230784,close,1
ZRX/USDT:USDT,26.0,open_long,close_long,0.444200,0.44320,0.443100,Trade_nb_7,Trade_nb_7_slV_1,1117610395337039872,1117610398554066945,close,1
PYTH/USDT:USDT,25.0,open_short,close_short,0.460900,0.46190,0.462300,Trade_nb_13,Trade_nb_13_slV_1,1117610685561905152,1117610688707633152,close,1


,amount,side,sl_side,opening_price,sl_triggerPrice,sl_Execut_Price,clientOid,sl_clientOid,id,sl_id,status,sl_version
symbol,,,,,,,,,,,,
SXP/USDT:USDT,28.80,open_short,close_short,0.40670,0.40740,0.40750,Trade_nb_1,Trade_nb_1_slV_4,1117609958521245696,1117610111130996736,Open,4
MATIC/USDT:USDT,12.00,open_long,close_long,0.91010,0.91060,0.91050,Trade_nb_4,Trade_nb_4_slV_5,1117610216579993600,1117610737160228864,Open,5
BSV/USDT:USDT,0.22,open_long,close_long,52.28000,52.23000,52.22000,Trade_nb_6,Trade_nb_6_slV_5,1117610353490468864,1117610887739936768,Open,5
EOS/USDT:USDT,13.00,open_long,close_long,0.85310,0.85180,0.85170,Trade_nb_8,Trade_nb_8_slV_2,1117610403272658944,1117610540438986752,Open,2
AR/USDT:USDT,1.40,open_short,close_short,8.28100,8.29200,8.29800,Trade_nb_9,Trade_nb_9_slV_1,1117610411887759360,1117610415025102848,Open,1
RVN/USDT:USDT,488.00,open_long,close_long,0.02409,0.02403,0.02400,Trade_nb_10,Trade_nb_10_slV_1,1117610472885526528,1117610476408737792,Open,1
AGLD/USDT:USDT,11.70,open_long,close_long,1.00530,1.00440,1.00430,Trade_nb_11,Trade_nb_11_slV_2,1117610481261551616,1117610746618384385,Open,2
DOT/USDT:USDT,1.00,open_long,close_long,7.29100,7.28600,7.27900,Trade_nb_12,Trade_nb_12_slV_1,1117610614447476736,1117610617668706304,Open,1
WAXP/USDT:USDT,157.00,open_long,close_long,0.07476,0.07463,0.07462,Trade_nb_14,Trade_nb_14_slV_1,1117610771482222592,1117610774665699329,Open,1


CancelledError: 